In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt 


In [33]:
num_pat_list = [] #10 patients coming in for 10 weeks
for i in range(10):
    num_pat_list.append(10)
sp1=10
sp2=3
num_pats=sum(num_pat_list) #total is 100 patients

#setting the seed
np.random.seed(2019)

#currently we assume that the patient needs three collections
coll = []
for i in range(num_pats):
    coll.append(3)
coll= np.array(coll) 


In [34]:
#build a schedule for doctors
doctor = []
for i in range(1):
    availability = []
    for j in range(60): #put the total days in place of the number here.
        availability.append(np.random.randint(0,2)) #they are randomly available on each day.
    doctor.append(availability)
print(doctor)
len(doctor)

[[0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1]]


1

In [35]:
def make_schedule_d1(num_patients,collections, max_space1, max_days):
    #initialize schedules for d1 procedure.  These record the number of patients scheduled for each procedure on each day.
    space1 = np.zeros(max_days)
    
    n1=len(space1)
    
    #initialize a vector which will keep track of each patient's schedule for d1.
    patients_d1 = []
    for i in range(num_patients):
        for j in range(n1):
            #for each patient, we will schedule them in the first available
            #time slot for the d1 procedure.
            if space1[j]< max_space1: #if there is space in the d1 facility on day j (interpret weeks as 5 days long, 1 is )
                for k in range(collections[i]):
                    space1[j+k]=space1[j+k]+1 #add patient i to the schedule on day k for each day that they are collecting.
                patients_d1.append(j) #record the fact that patient i was scheduled for d1 on day j
                break
           
    return(np.array(patients_d1))

d1_pat_schedule = make_schedule_d1(num_pats,coll, sp1, 60)
d1_pat_schedule

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 15,
       15, 15, 15, 15, 15, 15, 15, 15, 15, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 24, 24, 24, 24, 24,
       24, 24, 24, 24, 24, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27])

In [36]:
def make_schedule_d2_doctors(d1_schedule, collections,doc_sched, max_space2, max_days):
    space2 = np.zeros(max_days) #the length of this list is the max number of days 
    patients2=[] #Initiate a patient list
    #make a list assigning a patient to one of the doctors
    patient_doc = []
    for i in range(len(d1_schedule)):
        patient_doc.append(np.random.randint(0,len(doc_sched))) #randomly assigns each patient a doctor
    for j, day in enumerate(d1_schedule): #j is keeping track of patients
        for k in range(day,max_days-collections[j]): #k is keeping track of the day
            #print(space2[k+collections[j]], doc_sched[patient_doc[j]][k])
            if space2[k+collections[j]]<8*max_space2 and doc_sched[patient_doc[j]][k]==1: #if there is space and the doc is available
                space2[k+collections[j]]+=1
                patients2.append(k+collections[j])
                break
            
    return(np.array(patients2))



In [37]:
#Monday- 0 , Tuesday-1, Wednesday-2, Thursday-4, Friday-4, Monday-7
d2_pat_schedule = make_schedule_d2_doctors(d1_pat_schedule, coll,doctor, sp2, 60)
d2_pat_schedule

array([ 5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 19,
       19, 19, 19, 19, 19, 19, 19, 19, 19, 24, 24, 24, 24, 24, 24, 24, 24,
       24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32])

In [38]:
#Let us compute the wait time
def wait_time(d1_schedule, d2_schedule, collections):
    q1 = []
    n = len(d1_schedule)
    for i in range(n):
        for j in range(collections[i]):
            if (d1_schedule[i]+j)%5==4: #if any of the collection days is a Friday
                q1.append(d2_schedule[i]-(d1_schedule[i]+collections[i])+2) #add 2 to the wait time.
                break
            #if none of the collection days is a friday, then the wait time is the difference between the
            #d1 end day plus 1 and the d2 start day.  A day of waiting is a day where nothing else happens
            elif j==collections[i]-1:
                q1.append(d2_schedule[i]-(d1_schedule[i]+collections[i]))
    return q1
    
print(wait_time(d1_pat_schedule,d2_pat_schedule,coll))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
